### Mount Google Drive to runtime

In [1]:
from google.colab import drive
from os.path import join

# Mounting location on runtime for GDrive
ROOT = '/content/drive'

# Project workspace on GDrive
PROJECT_PATH = 'My Drive/Github'

# Mount GDrive on the runtime
drive.mount(ROOT)

# Create the full runtime project path and create a workspace at that location
WORKING_PATH = join(ROOT, PROJECT_PATH)
!mkdir "{WORKING_PATH}" 
%cd "{WORKING_PATH}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Github’: File exists
/content/drive/My Drive/Github


### Import libraries

In [ ]:
import json
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

f = open('/content/drive/My Drive/Github/mlteam4/sandbox/Amazon_Topic_Attributes_20200617061621.json')
topicDict = json.load(f)
f.close()

topics = list(topicDict.keys())

topicFeatures = []
labels = []

count = 0
for topic in topics:
    # Combine topic title and comments into one string
    title = topicDict[topic]['Topic Title']
    leadingComment = topicDict[topic]['Leading Comment']
    #otherComments = topicDict[topic]['Other Comments']
    
    commentList = [title] + [leadingComment] 
    featureString = ' '.join(commentList)

    # Replace newline and tab characters with spaces
    featureString = featureString.replace('\n', ' ')
    featureString = featureString.replace('\t', ' ')

    # Convert all letters to lowercase
    featureString = featureString.lower()

    table = str.maketrans('', '', string.punctuation)
    #featureString = featureString.translate(table)

    #featureString = featureString.encode(encoding='ascii', errors='ignore').decode('ascii')

    wordList = featureString.split()

    stop_words = set(stopwords.words('english'))
    wordList = [word for word in wordList if not word in stop_words]

    wordList = [word for word in wordList if is_ascii(word)]

    punctuation = '!"#%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    wordList = [word.strip(punctuation) for word in wordList]

    wordList = ['########' if (word.replace('.','').isdigit()) \
                else '$$$$$$$$' if (word.replace('.','').replace('$','').isdigit()) \
                else word \
                for word in wordList]

    #print(wordList)
    #wordList = ['########' if (word.replace('.','').isdigit()) else word for word in wordList]
    #wordList = ['########' if (word.translate(table).isdigit()) else word for word in wordList]
    

    #wordList = [word for word in wordList if word.isalpha()]

    

    #table = str.maketrans('', '', string.punctuation)
    #wordList = [w.translate(table) for w in wordList]

    #print(wordList)

    featureString = ' '.join(wordList)
    if (featureString.strip() == ''):
        continue
    print(count)
    count += 1
    print(featureString)

    topicFeatures.append(featureString)

    # Append topic category to ground truth labels
    labels.append(topicDict[topic]['Category'])



vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(topicFeatures)

#X = 1000 * X

print(X.toarray())
print(X.shape)
#print(len(vectorizer.get_feature_names()))

#print(labels)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1)
#X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=8)
#print(X_test.shape)

multinominalClassifier = MultinomialNB()
multinominalClassifier.fit(X_train, y_train)

numCorrect = (multinominalClassifier.predict(X_test) == y_test).sum()
accuracy = numCorrect / len(y_test)
print('Multinominal Classifier Accuracy: ' + str(accuracy))

gaussianClassifier = GaussianNB()
gaussianClassifier.fit(X_train.toarray(), y_train)

numCorrect = (gaussianClassifier.predict(X_test.toarray()) == y_test).sum()
accuracy = numCorrect / len(y_test)
print('Gausian Classifier Accuracy: ' + str(accuracy))



Output hidden; open in https://colab.research.google.com to view.